In [12]:
import pandas as pd

filename = "aisdk-2023-11-08-6xs_2_kkn"
#filename = "aisTestFile_500k_kkn"
#filename = "aisdk-2023-11-11_266288000_s_1_kkn_Step2b"

# data =  pd.read_csv("../workdir/AIS-KNN-Files/aisdk-2023-11-08-xs_2_kkn.csv", lineterminator='$').values.reshape((-1,100,6))
data =  pd.read_csv("../workdir/AIS-KNN-Files/"+filename+".csv", lineterminator='$').values.reshape((-1,100,6))

#data =  pd.read_csv("../workdir/AIS-KNN-Files/aisdk-2023-11-08-xs_2_kkn.csv", lineterminator='$').values.reshape((-1,100,6))

In [13]:
# https://github.com/sktime/sktime/blob/v0.11.4/sktime/datatypes/_panel/_convert.py#L608-L668
multiindex = pd.MultiIndex.from_product([range(s) for s in data.shape], names=["track", "timeindex", "column"])
data = pd.Series(data.flatten(), index=multiindex).unstack(level="column")
data.columns = ["mmsi", "time", "x", "y", "deg", "dist"]
data = data.drop(columns=["deg", "dist"])
data["time"] = pd.to_datetime(data["time"], unit="s")
train_ships = pd.Series(data.mmsi.unique()).sample(frac=0.7, random_state=42)
train_ships = pd.Series(data.mmsi.unique()).sample(frac=1, random_state=42)
train = data[data.mmsi.isin(train_ships)].drop(columns="mmsi")
test = data[~data.mmsi.isin(train_ships)].drop(columns="mmsi")
from joblib import Parallel, delayed

In [14]:
train.head(200)

time       x       y
track timeindex                                    
0     0         2023-11-07 23:00:01  1015.7 -2582.5
      1         2023-11-07 23:00:03  1014.1 -2573.3
      2         2023-11-07 23:00:04  1014.1 -2573.3
      3         2023-11-07 23:00:06  1011.9 -2564.0
      4         2023-11-07 23:00:07  1011.9 -2564.0
...                             ...     ...     ...
1     95        2023-11-07 23:02:44   926.7  2268.8
      96        2023-11-07 23:02:46   927.3  2266.8
      97        2023-11-07 23:02:47   927.3  2266.8
      98        2023-11-07 23:02:50   928.9  2264.3
      99        2023-11-07 23:02:51   928.9  2264.3

[200 rows x 3 columns]

In [15]:
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np

def generate_row(df):
    df = df.copy()
    df["src"] = "output"
    df["time"] = (df["time"].max()-df["time"]).dt.total_seconds()
    df["x"] = df["x"] - df["x"].iloc[0]
    df["y"] = df["y"] - df["y"].iloc[0]
    # df['deltaX'] = pd.Series(df["x"].iloc[0], index=df.index)
    # df['deltaY'] = pd.Series(df["y"].iloc[0], index=df.index)
    # df['deltaTime'] = pd.Series(test["time"].max(), index=df.index)
    df.iloc[:-1,-1] = "input_" + pd.Series(np.arange(df.shape[0]-1)).astype(str)
    df=df.melt(id_vars="src")
    df["colname"] = df.src + "_" + df.variable
    df=df.drop(columns=["src", "variable"]).set_index("colname")
    #df=df.drop(columns=["timeindex"]).set_index("colname")
    df=df.transpose().reset_index(drop=True)
    df.columns.name = None
    df=df.sort_index(axis=1).drop(["output_time"], axis=1)
    df=df.loc[:, ~df.columns.to_series().str.endswith('index')]
    #df=df.drop(columns.to_series().str.ends_with('index'))

    return df

def generate_model_frame(df, knowns):
    window_size = knowns + 1
    skipped_windows = 4

    windows = df.reset_index(drop=False).groupby('track').rolling(window_size)
    windows = (win for i, win in enumerate(windows) if len(win) == window_size and i % (skipped_windows + 1) == 0)
    windows = list(windows)
    res = Parallel(n_jobs=-1)(delayed(generate_row)(win) for win in tqdm(windows))
    #res = (generate_row(win) for win in tqdm(windows))
    return pd.concat(res, ignore_index=True)


In [ ]:
from pathlib import Path
targetrows_knowns = 3, 6, 9, 12
for target in targetrows_knowns:
    path = Path(f"../workdir/AIS-ModelFrames/{filename}_train_{target}_{0}.csv")
    if not path.exists():     
        train_model = generate_model_frame(train, target)
        #train_model.head()
        test_model = generate_model_frame(test, target)
        train_model.to_csv(path, index=False)
        test_model.to_csv(f"../workdir/AIS-ModelFrames/{filename}_test_{target}_{0}.csv", index=False)
        print(f"Done with {target}")

## All in One

In [1]:
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np
from pathlib import Path
import pandas as pd

filename = "aisdk-2023-11-08_1_kkn"
#filename = "aisdk-2023-11-08-5xs_1_kkn"
#filename = "aisdk-2023-11-08-s_3_kkn"
#filename = 'aisdk-2023-11-08-xs_3_kkn'
#filename = "aisdk-2023-11-08-s_2_kkn"
#filename = "aisdk-2023-11-08-6xs_2_kkn"
#filename = "aisTestFile_500k_kkn"
#filename = "aisdk-2023-11-11_266288000_s_1_kkn_Step2b"

# data =  pd.read_csv("../workdir/AIS-KNN-Files/aisdk-2023-11-08-xs_2_kkn.csv", lineterminator='$').values.reshape((-1,100,6))
data =  pd.read_csv(f"../workdir/AIS-KNN-Files/{filename}.csv", lineterminator='$').values.reshape((-1,100,6))

# https://github.com/sktime/sktime/blob/v0.11.4/sktime/datatypes/_panel/_convert.py#L608-L668
multiindex = pd.MultiIndex.from_product([range(s) for s in data.shape], names=["track", "timeindex", "column"])
data = pd.Series(data.flatten(), index=multiindex).unstack(level="column")
data.columns = ["mmsi", "time", "x", "y", "deg", "dist"]
data = data.drop(columns=["deg", "dist"])
data["time"] = pd.to_datetime(data["time"], unit="s")
train_ships = pd.Series(data.mmsi.unique()).sample(frac=0.7, random_state=42)
train = data[data.mmsi.isin(train_ships)].drop(columns="mmsi")
test = data[~data.mmsi.isin(train_ships)].drop(columns="mmsi")

data = []
train_ships = []

def generate_row(df):
    df = df.copy()
    df["src"] = "output"
    df["time"] = (df["time"].max()-df["time"]).dt.total_seconds()
    df["x"] = df["x"] - df["x"].iloc[0]
    df["y"] = df["y"] - df["y"].iloc[0]
    df.iloc[:-1,-1] = "input_" + pd.Series(np.arange(df.shape[0]-1)).astype(str)
    df=df.melt(id_vars="src")
    df["colname"] = df.src + "_" + df.variable
    df=df.drop(columns=["src", "variable"]).set_index("colname")
    df=df.transpose().reset_index(drop=True)
    df.columns.name = None
    df=df.sort_index(axis=1).drop(["output_time"], axis=1)
    df=df.loc[:, ~df.columns.to_series().str.endswith('index')]
    return df

def generate_model_frame(df, known):
    window_size = known + 1
    skipped_windows = 4

    windows = df.reset_index(drop=False).groupby('track').rolling(window_size)
    windows = (win for i, win in enumerate(windows) if len(win) == window_size and i % (skipped_windows + 1) == 0)
    windows = list(windows)
    res = Parallel(n_jobs=-1)(delayed(generate_row)(win) for win in tqdm(windows))
    return pd.concat(res, ignore_index=True)



# var = list(test.shape)
# var = var[0]
# print(f"Test: {var}")
# testlimit = var * 7 * 0.2
# print(f"Testlimit: {testlimit}")

# var = list(train.shape)
# var = var[0]
# print(f"Train: {var}")
# trainlimit = var * 7 * 0.2
# print(f"Trainlimit: {trainlimit}")

#targetrows_knowns = 3, 6, 9, 12
#for target in targetrows_knowns:

for known in range(12, 2, -1):
    path = Path(f"../workdir/AIS-ModelFrames/{filename}_train_{known}_0.csv")
    if not path.exists():     
        train_model = generate_model_frame(train, known)
        test_model = generate_model_frame(test, known)

        if known == 12:   
            var = list(train_model.shape)
            var = var[0]
            trainlimit = var
            print(f"Trainlimit: {trainlimit}")

            var = list(test_model.shape)
            var = var[0]
            testlimit = var
            print(f"Testlimit: {testlimit}")

        print(train_model.shape)
        train_model = train_model.sample(n = trainlimit)
        print(train_model.shape)

        print(test_model.shape)
        test_model = test_model.sample(n = testlimit)
        print(test_model.shape)
        
        train_model.to_csv(path, index=False)
        test_model.to_csv(f"../workdir/AIS-ModelFrames/{filename}_test_{known}_0.csv", index=False)
        print(f"Done with {known}")

  0%|          | 20/1112344 [00:00<10:29:15, 29.46it/s]/home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
100%|██████████| 1112344/1112344 [12:13<00:00, 1516.99it/s]


Trainlimit: 2681903
Testlimit: 1112344
(2681903, 38)
(2681903, 38)
(1112344, 38)
(1112344, 38)
Done with 12


100%|██████████| 1112344/1112344 [11:55<00:00, 1553.94it/s]


(2681903, 35)
(2681903, 35)
(1112344, 35)
(1112344, 35)
Done with 11


100%|██████████| 1177776/1177776 [12:36<00:00, 1556.22it/s]


...

(2997421, 17)
(2681903, 17)
(1243208, 17)
(1112344, 17)
Done with 5


100%|██████████| 1243208/1243208 [13:16<00:00, 1560.73it/s]


(2997421, 14)
(2681903, 14)
(1243208, 14)
(1112344, 14)
Done with 4


100%|██████████| 1243208/1243208 [13:15<00:00, 1563.41it/s]


(2997421, 11)
(2681903, 11)
(1243208, 11)
(1112344, 11)
Done with 3


In [16]:
import numpy as np
import pandas as pd

filename = "aisdk-2023-11-08-xs_3_kkn"

noise = 0
known = 6
limit = 200
# for known in range(3, 13, 1):
path = Path(f"../workdir/AIS-ModelFrames/{filename}_test_{known}_{noise}.csv") 
#test_model.head(103)
print(test_model.shape)

tmp_test_model = test_model[:limit].copy()
print(tmp_test_model.shape)

test_model = []

AttributeError: 'list' object has no attribute 'shape'

In [7]:
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np

test = test.copy()
test["src"] = "output"
test["time"] = (test["time"].max()-test["time"]).dt.total_seconds()
test['e'] = pd.Series((test["time"].max()) , index=test.index)
test.head(100)

NameError: name 'test' is not defined

## Resample Idea (nicht weiter verfolgt)

In [7]:
time_resample = "10s"
resampled = example.resample(time_resample, level="time").mean().interpolate(method="time")
resampled

NameError: name 'example' is not defined

In [114]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(example['x'], example['y'], label='Example')
plt.plot(resampled['x'], resampled['y'], label=f'Resampled {time_resample}')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('X and Y Coordinates Through Time')
plt.legend()
plt.show()

In [112]:
example

In [124]:
X_resampled = X.reset_index().drop(columns=["epoch_time", "degree", "distance", "timepoint"]).set_index(['track','time','mmsi'])
X_resampled = X_resampled.groupby(["track","mmsi"]).apply(lambda df: df.resample(time_resample, level="time").mean().interpolate(method="time"))
X_resampled.head()

In [126]:
import gc
gc.collect()

In [127]:
X_resampled.groupby("track").size().describe()

Ziel: Bei Zeit 0

Beispiele für 3:
1. -5s, -10s, -15s:
2. -5s, -10s, -20s:
3. -30s, -60s, -61s:

In [ ]:
target_y, target_x               , old5s_x, old5s_y, old10s_x, old10s_y

In [ ]:
target_y, target_x,           old5s_x, old5s_y, old10s_x, old10s_y, old15s_x, old15s_y, old20s_x, old20s_y,

In [ ]:
9 Eingänge
8 hidden Neuron
2 Ausgänge


In [278]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm.auto import tqdm

In [279]:
# https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
data_in = pd.read_csv("../workdir/AIS-KNN-Files/aisdk-2023-11-08-xs_2_kkn.csv", lineterminator="$", header=None)

# KKN Data Analysis

In [280]:
# (bewegung, zeitpunkte, daten)
resshaped = data_in.values.reshape((-1, 100, 6))
# resshaped[0,0,:]
# X,Y and Degree,Distance are the same only different coordinate systems
# MMSI,Time,X,Y,Degree,Distance

In [281]:
resshaped.shape

In [282]:
resshaped.shape

In [283]:
tracks = range(resshaped.shape[0])
timepoints = range(resshaped.shape[1])
columns = range(resshaped.shape[2])

# stolen from https://github.com/sktime/sktime/blob/v0.11.4/sktime/datatypes/_panel/_convert.py#L608-L668
index = pd.MultiIndex.from_product([tracks, timepoints, columns], names=['track', 'timepoint', 'column'])
X = pd.Series(resshaped.flatten(), index=index)
X = X.unstack(level='column')
X.columns = ['mmsi', 'epoch_time', 'x', 'y', 'degree', 'distance']
X['time'] = pd.to_datetime(X['epoch_time'], unit='s')

In [284]:
# Time Ranges
X.groupby('track')["time"].diff().dt.total_seconds().describe()

In [285]:
secs = X.groupby('track')["time"].diff().dt.total_seconds()
_ = sns.histplot(secs, binwidth=1)

In [286]:
example = X.query("track == 0").copy()
example.time = example.time - example.time.min()
example = example.reset_index().set_index(['track','time','mmsi']).drop(columns=["epoch_time", "degree", "distance", "timepoint"])

1. 3 Bekannte, 1 Ziel
2. 6 Unbekannte, 1 Ziel
3. 12 Unbekannte, 1 Ziel